# Bigger Dataset - EDA 1: Champkoi & GCKoi

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from openai import OpenAI
from pydantic import BaseModel

In [4]:
# import data

champkoi = pd.read_csv('../data/champkoi_data.csv')
gckoi = pd.read_csv('../data/gckoi_data.csv')



# Data from Champkoi - 185 pictures

In [5]:
# check the info

champkoi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   185 non-null    object 
 1   sku          185 non-null    int64  
 2   variety      185 non-null    object 
 3   title        185 non-null    object 
 4   price        185 non-null    float64
 5   in_stock     185 non-null    bool   
 6   link         185 non-null    object 
 7   description  0 non-null      float64
 8   category     185 non-null    object 
 9   tags         185 non-null    object 
 10  image_url    185 non-null    object 
 11  image_path   185 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(8)
memory usage: 16.2+ KB


In [6]:
# check the data

champkoi.head()

,Unnamed: 0,sku,variety,title,price,in_stock,link,description,category,tags,image_url,image_path
0,87e22e,9689837273400,"A: 14"" (36cm) Female Showa [Isa]","A: 14"" (36cm) Female Showa [Isa] — 2025WINTER-012",450.0,False,https://www.champkoi.com/products/2025winter-012,NaN,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/87e22e_A_14_(36cm)_Female_Showa_...
1,188b5a,9689846350136,"A: 14"" (36cm) Male Showa [Isa]","A: 14"" (36cm) Male Showa [Isa] — 2025WINTER-014",450.0,True,https://www.champkoi.com/products/2025winter-014,NaN,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/188b5a_A_14_(36cm)_Male_Showa_[I...
2,911317,9689850118456,"A: 13"" (33cm) Male Showa [Isa]","A: 13"" (33cm) Male Showa [Isa] — 2025WINTER-015",450.0,True,https://www.champkoi.com/products/2025winter-015,NaN,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/911317_A_13_(33cm)_Male_Showa_[I...
3,dd27e7,9689853329720,"A: 14"" (36cm) Female Showa [Isa]","A: 14"" (36cm) Female Showa [Isa] — 2025WINTER-016",450.0,False,https://www.champkoi.com/products/2025winter-016,NaN,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/dd27e7_A_14_(36cm)_Female_Showa_...
4,de2cb4,9689970475320,"A: 14"" (36cm) Male Kohaku [Marusaka]","A: 14"" (36cm) Male Kohaku [Marusaka] — 2025WIN...",450.0,False,https://www.champkoi.com/products/2025winter-017,NaN,Group_Koi,"['Kohaku', 'Shiro Utsuri', 'Shusui']",https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/de2cb4_A_14_(36cm)_Male_Kohaku_[...


In [7]:
# split the title string to extract details

champkoi['split'] = champkoi['title'].apply(lambda x: x.split())

In [8]:
champkoi['split']

0      [A:, 14", (36cm), Female, Showa, [Isa], —, 202...
1      [A:, 14", (36cm), Male, Showa, [Isa], —, 2025W...
2      [A:, 13", (33cm), Male, Showa, [Isa], —, 2025W...
3      [A:, 14", (36cm), Female, Showa, [Isa], —, 202...
4      [A:, 14", (36cm), Male, Kohaku, [Marusaka], —,...
                             ...                        
180                 [Default, Title, —, 25D-024, Kujaku]
181     [Default, Title, —, 2024FALL-080, Mukashi, Ogon]
182          [Default, Title, —, 25D-031, Tancho, Showa]
183    [A:, 32", Female, Kohaku, [Nogami], —, 2023FAL...
184    [Default, Title, —, 2024FALL-127, Yellow, Muka...
Name: split, Length: 185, dtype: object

In [9]:
champkoi['split'][0]

['A:', '14"', '(36cm)', 'Female', 'Showa', '[Isa]', '—', '2025WINTER-012']

In [10]:
# count the strings to see which terms are most frequently used, to help distinguish which varieties are in this dataset

counts = {}
for fish in champkoi['split']:
  for item in fish:
    counts[item] = counts.get(item, 0) + 1

In [11]:
# see the word count

df = pd.DataFrame(list(counts.items()), columns=['item', 'count'])
df = df.sort_values(by='count', ascending=False)
df.head(30)

,item,count
6,—,185
81,Default,150
82,Title,150
4,Showa,43
14,Kohaku,37
0,A:,35
44,Ginrin,25
3,Female,21
20,Golden,17
21,Corn,17


- remove dashes, 'Default', 'Title', 'A:' and anything that starts with a number, or open parenthesis
- Showa, Kohaku, Bekko are all koi fish varieties
- Extract additional features including: Female/Male
- Extract trait information, such as: Golden, Ginrin, Doitsu
- Get the fish length out as a separate feature, convert to either inches or cm to be consistent

In [12]:
champkoi['split'][0]

['A:', '14"', '(36cm)', 'Female', 'Showa', '[Isa]', '—', '2025WINTER-012']

In [13]:
# Remove '—', 'Default', 'Title', 'A:'

strings_to_remove = ['—', 'Default', 'Title', 'A:']

new_list = [item for item in champkoi['split'][0] if item not in strings_to_remove]
new_list

['14"', '(36cm)', 'Female', 'Showa', '[Isa]', '2025WINTER-012']

In [14]:
# Create a function to remove the strings

def remove_strings(list):
  new_list = [item for item in list if item not in strings_to_remove]
  return new_list

In [15]:
# Apply the string removal to all items

champkoi['cleaned_strings'] = champkoi['split'].apply(remove_strings)

In [16]:
# check the cleaned strings

champkoi['cleaned_strings']

0      [14", (36cm), Female, Showa, [Isa], 2025WINTER...
1      [14", (36cm), Male, Showa, [Isa], 2025WINTER-014]
2      [13", (33cm), Male, Showa, [Isa], 2025WINTER-015]
3      [14", (36cm), Female, Showa, [Isa], 2025WINTER...
4      [14", (36cm), Male, Kohaku, [Marusaka], 2025WI...
                             ...                        
180                                    [25D-024, Kujaku]
181                        [2024FALL-080, Mukashi, Ogon]
182                             [25D-031, Tancho, Showa]
183        [32", Female, Kohaku, [Nogami], 2023FALL-109]
184                [2024FALL-127, Yellow, Mukashi, Ogon]
Name: cleaned_strings, Length: 185, dtype: object

In [17]:
# check start with parenthesis to put into a 'size in cm' column

champkoi['cleaned_strings'][0]

['14"', '(36cm)', 'Female', 'Showa', '[Isa]', '2025WINTER-012']

In [18]:
for item in champkoi['cleaned_strings'][0]:
  if item.startswith('('):
    # remove parenthesis
    print(item.strip('()'))

36cm


In [19]:
# create a function to check for parenthesis and strip parenthesis and return in new column

def check_for_parenthesis(list):
  for item in list:
    if item.startswith('('):
      return item.strip('()')

In [20]:
# use the above function to create a new column with the length in cm

champkoi['length_cm'] = champkoi['cleaned_strings'].apply(check_for_parenthesis)
champkoi['length_cm']

0      36cm
1      36cm
2      33cm
3      36cm
4      36cm
       ... 
180    None
181    None
182    None
183    None
184    None
Name: length_cm, Length: 185, dtype: object

In [21]:
# check for 1 double-quote to extract the length in inches

for item in champkoi['cleaned_strings'][0]:
  if '"' in item:
    print(True)
  else:
    print(False)

True
False
False
False
False
False


In [22]:
# create a function to extract the length in inches

def get_inches(list):
  for item in list:
    if '"' in item:
      return item

In [23]:
# apply the function to create a new column

champkoi['length_inches'] = champkoi['cleaned_strings'].apply(get_inches)
champkoi['length_cm']

0      36cm
1      36cm
2      33cm
3      36cm
4      36cm
       ... 
180    None
181    None
182    None
183    None
184    None
Name: length_cm, Length: 185, dtype: object

In [24]:
champkoi['length_inches'].value_counts()

length_inches
15"       8
16"       8
14"       6
17"       4
13"       3
18"       2
14-15"    1
20-22"    1
22"       1
32"       1
Name: count, dtype: int64

- there are ranges in the length that will need to be addressed, but for now, this indicates that the image may contain multiple fish. I will keep it as a range to signal the fact that the picture contains more than 1 fish, in case I choose to use this method to remove these sample from the data later.

In [25]:
# extract the fish gender

for item in champkoi['cleaned_strings'][0]:
  if item.lower() == 'female' or item.lower() == 'male':
    print(item)

Female


In [26]:
# create a function to extract the fish gender

def get_gender(list):
  for item in list:
    if item.lower() == 'female' or item.lower() =='male':
      return item

In [27]:
# apply the get gender function to create a new column

champkoi['gender'] = champkoi['cleaned_strings'].apply(get_gender)
champkoi['gender']

0      Female
1        Male
2        Male
3      Female
4        Male
        ...  
180      None
181      None
182      None
183    Female
184      None
Name: gender, Length: 185, dtype: object

In [28]:
# the breeder name is in bracket, extract the breeder name

for item in champkoi['cleaned_strings'][0]:
  if item.startswith('['):
    print(item.strip('[]'))

Isa


In [29]:
# create a function to extract the breeder name

def get_breeder(list):
  for item in list:
    if item.startswith('['):
      return item.strip('[]')

In [30]:
# apply the function to create a new column for the breeder name

champkoi['breeder'] = champkoi['cleaned_strings'].apply(get_breeder)
champkoi['breeder']

0           Isa
1           Isa
2           Isa
3           Isa
4      Marusaka
         ...   
180        None
181        None
182        None
183      Nogami
184        None
Name: breeder, Length: 185, dtype: object

In [31]:
# description column is empty, drop column

champkoi = champkoi.drop(columns='description')

In [32]:
# check the tags column

champkoi['tags'][4]

"['Kohaku', 'Shiro Utsuri', 'Shusui']"

In [33]:
# check the type

type(champkoi['tags'][4])

str

In [34]:
# change the string to a list of strings

for item in champkoi['tags'][4].strip('"').strip('[]').split(', '):
  print([item.strip('"').strip("'")])

['Kohaku']
['Shiro Utsuri']
['Shusui']


In [35]:
# check if it works on only 1 string

for item in champkoi['tags'][0].strip('"').strip('[]').split(', '):
  print([item.strip('"').strip("'")])

['Showa']


In [36]:
champkoi['tags'][184]

"['Yellow Mukashi Ogon']"

In [37]:
for item in champkoi['tags'][0].strip('"').strip('[]').split(', '):
  print([item.strip('"').strip("'")])

['Showa']


In [38]:
# create a function to change string of a list into a list of strings

def string_to_list(string):
  list = []
  for item in string.strip('"').strip('[]').split(', '):
    list.append(item.strip('"').strip("'"))
  return list

In [39]:
# used the function to create a new column that has all the tags in a list of strings

champkoi['tags_list'] = champkoi['tags'].apply(string_to_list)
champkoi['tags_list']

0                             [Showa]
1                             [Showa]
2                             [Showa]
3                             [Showa]
4      [Kohaku, Shiro Utsuri, Shusui]
                    ...              
180                          [Kujaku]
181                    [Mukashi Ogon]
182                    [Tancho Showa]
183                          [Kohaku]
184             [Yellow Mukashi Ogon]
Name: tags_list, Length: 185, dtype: object

In [40]:
# check the value counts

champkoi['tags_list'].value_counts()

tags_list
[Kohaku]                                      31
[Showa]                                       25
[Golden Corn]                                 15
[Sanke]                                       10
[Ginrin Showa]                                 7
                                              ..
[Doitsu Kohaku, Ki Utsuri, Kin Showa]          1
[Kin Showa, Kohaku, Tancho Sanke]              1
[Beni Ginga, Doitsu Kohaku, Platinum Ogon]     1
[Dragon Scale, Ochiba]                         1
[Yellow Mukashi Ogon]                          1
Name: count, Length: 83, dtype: int64

In [41]:
# rename column to uuid

champkoi = champkoi.rename(columns={'Unnamed: 0':'uuid'})
champkoi

,uuid,sku,variety,title,price,in_stock,link,category,tags,image_url,image_path,split,cleaned_strings,length_cm,length_inches,gender,breeder,tags_list
0,87e22e,9689837273400,"A: 14"" (36cm) Female Showa [Isa]","A: 14"" (36cm) Female Showa [Isa] — 2025WINTER-012",450.0,False,https://www.champkoi.com/products/2025winter-012,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/87e22e_A_14_(36cm)_Female_Showa_...,"[A:, 14"", (36cm), Female, Showa, [Isa], —, 202...","[14"", (36cm), Female, Showa, [Isa], 2025WINTER...",36cm,"14""",Female,Isa,[Showa]
1,188b5a,9689846350136,"A: 14"" (36cm) Male Showa [Isa]","A: 14"" (36cm) Male Showa [Isa] — 2025WINTER-014",450.0,True,https://www.champkoi.com/products/2025winter-014,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/188b5a_A_14_(36cm)_Male_Showa_[I...,"[A:, 14"", (36cm), Male, Showa, [Isa], —, 2025W...","[14"", (36cm), Male, Showa, [Isa], 2025WINTER-014]",36cm,"14""",Male,Isa,[Showa]
2,911317,9689850118456,"A: 13"" (33cm) Male Showa [Isa]","A: 13"" (33cm) Male Showa [Isa] — 2025WINTER-015",450.0,True,https://www.champkoi.com/products/2025winter-015,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/911317_A_13_(33cm)_Male_Showa_[I...,"[A:, 13"", (33cm), Male, Showa, [Isa], —, 2025W...","[13"", (33cm), Male, Showa, [Isa], 2025WINTER-015]",33cm,"13""",Male,Isa,[Showa]
3,dd27e7,9689853329720,"A: 14"" (36cm) Female Showa [Isa]","A: 14"" (36cm) Female Showa [Isa] — 2025WINTER-016",450.0,False,https://www.champkoi.com/products/2025winter-016,Group_Koi,['Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/dd27e7_A_14_(36cm)_Female_Showa_...,"[A:, 14"", (36cm), Female, Showa, [Isa], —, 202...","[14"", (36cm), Female, Showa, [Isa], 2025WINTER...",36cm,"14""",Female,Isa,[Showa]
4,de2cb4,9689970475320,"A: 14"" (36cm) Male Kohaku [Marusaka]","A: 14"" (36cm) Male Kohaku [Marusaka] — 2025WIN...",450.0,False,https://www.champkoi.com/products/2025winter-017,Group_Koi,"['Kohaku', 'Shiro Utsuri', 'Shusui']",https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/de2cb4_A_14_(36cm)_Male_Kohaku_[...,"[A:, 14"", (36cm), Male, Kohaku, [Marusaka], —,...","[14"", (36cm), Male, Kohaku, [Marusaka], 2025WI...",36cm,"14""",Male,Marusaka,"[Kohaku, Shiro Utsuri, Shusui]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,0db514,9876548682040,Default Title,Default Title — 25D-024 Kujaku,5800.0,True,https://www.champkoi.com/products/25d-024-kujaku,Single_Koi,['Kujaku'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/0db514_Default_Title_—_25D-024_K...,"[Default, Title, —, 25D-024, Kujaku]","[25D-024, Kujaku]",None,None,None,None,[Kujaku]
181,79245b,9610889953592,Default Title,Default Title — 2024FALL-080 Mukashi Ogon,6800.0,True,https://www.champkoi.com/products/2024fall-080...,Presale_Koi,['Mukashi Ogon'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/79245b_Default_Title_—_2024FALL-...,"[Default, Title, —, 2024FALL-080, Mukashi, Ogon]","[2024FALL-080, Mukashi, Ogon]",None,None,None,None,[Mukashi Ogon]
182,5a5805,9876550025528,Default Title,Default Title — 25D-031 Tancho Showa,6800.0,True,https://www.champkoi.com/products/25d-031-tanc...,Single_Koi,['Tancho Showa'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/5a5805_Default_Title_—_25D-031_T...,"[Default, Title, —, 25D-031, Tancho, Showa]","[25D-031, Tancho, Showa]",None,None,None,None,[Tancho Showa]
183,18f755,8815742452024,"A: 32"" Female Kohaku [Nogami]","A: 32"" Female Kohaku [Nogami] — 2023FALL-109",14000.0,True,https://www.champkoi.com/products/2023fall-109,Presale_Koi,['Kohaku'],https://cdn.shopify.com/s/files/1/0749/6132/22...,champkoi_data/18f755_A_32_Female_Kohaku_[Nogam...,"[A:, 32"", Female, Kohaku, [Nogami], —, 2023FAL...","[32"", Female, Kohaku, [Nogami], 2023FALL-109]",None,"32""",Female,Nogami,[Kohaku]


In [42]:
# # export df to csv

# champkoi.to_csv('champkoi_df.csv', index=False)

In [43]:
# export the dataframe to json to maintain list formats

with open('champkoi.json', 'w') as f:
  json.dump(champkoi.to_dict(orient='records'), f)

# Data from GCKoi - 408 pictures

In [44]:
# check the info

gckoi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   408 non-null    object 
 1   sku          408 non-null    int64  
 2   title        408 non-null    object 
 3   price        408 non-null    float64
 4   in_stock     408 non-null    bool   
 5   link         408 non-null    object 
 6   description  402 non-null    object 
 7   category     30 non-null     object 
 8   tags         408 non-null    object 
 9   image_url    408 non-null    object 
 10  image_path   408 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(8)
memory usage: 32.4+ KB


In [45]:
# check the data

gckoi.head()

,Unnamed: 0,sku,title,price,in_stock,link,description,category,tags,image_url,image_path
0,9718985,8428729860265,"GENJIRO GOSANKE 7""-9""",95.0,True,https://gckoi.com/products/genjiro-gosanke-7-9,"Breeder: Genjiro Kohaku, Sanke and ShowaSize: ...",NaN,"['genjiro', 'kohaku', 'koi', 'sanke', 'showa',...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/9718985_GENJIRO_GOSANKE_7-9.jpg
1,d9a7dbc,7856409346217,HIGH QUALITY SAKAI BLOODLINE TANCHO KOHAKU 24”...,3500.0,True,https://gckoi.com/products/high-quality-sakai-...,Breeder: MarukyuSize: 3sai 24”Sex: female,NaN,"['kohaku', 'koi', 'Marukyu', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/d9a7dbc_HIGH_QUALITY_SAKAI_BLOODLIN...
2,00d04b3,8291399860393,"HIROI DOITSU SANKE-SHOWA-TANCHO OCHIBA 14""-16""",600.0,False,https://gckoi.com/products/hiroi-doitsu-sanke-...,*SHIP END OF NOVEMBER*Breeder: HiroiSex: 4 fem...,Koi,"['fall 2024', 'HIROI', 'koi', 'nisai', 'ochiba...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/00d04b3_HIROI_DOITSU_SANKE-SHOWA-TA...
3,564bc0a,8305083449513,IKARASHI MIX BOWL #1,600.0,False,https://gckoi.com/products/ikarashi-mix-bowl-1,*SHIP END OF NOVEMBER*Breeder: IkarashiSex: 5 ...,NaN,"['Beni Kikokuryu', 'fall 2024', 'ikarashi', 'k...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/564bc0a_IKARASHI_MIX_BOWL_#1.png
4,88ec869,7993174229161,ISA KOHAKU GUARANTEE FEMALE WITH CERTIFICATE 2...,2500.0,True,https://gckoi.com/products/isa-showa-grow-out-...,"Breeder: Isa with certificateSize: 21.5""Sex: f...",NaN,"['isa', 'kohaku', 'koi', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/88ec869_ISA_KOHAKU_GUARANTEE_FEMALE...


In [46]:
# rename the uuid column appropriately

gckoi = gckoi.rename(columns={'Unnamed: 0': 'uuid'})
gckoi

,uuid,sku,title,price,in_stock,link,description,category,tags,image_url,image_path
0,9718985,8428729860265,"GENJIRO GOSANKE 7""-9""",95.0,True,https://gckoi.com/products/genjiro-gosanke-7-9,"Breeder: Genjiro Kohaku, Sanke and ShowaSize: ...",NaN,"['genjiro', 'kohaku', 'koi', 'sanke', 'showa',...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/9718985_GENJIRO_GOSANKE_7-9.jpg
1,d9a7dbc,7856409346217,HIGH QUALITY SAKAI BLOODLINE TANCHO KOHAKU 24”...,3500.0,True,https://gckoi.com/products/high-quality-sakai-...,Breeder: MarukyuSize: 3sai 24”Sex: female,NaN,"['kohaku', 'koi', 'Marukyu', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/d9a7dbc_HIGH_QUALITY_SAKAI_BLOODLIN...
2,00d04b3,8291399860393,"HIROI DOITSU SANKE-SHOWA-TANCHO OCHIBA 14""-16""",600.0,False,https://gckoi.com/products/hiroi-doitsu-sanke-...,*SHIP END OF NOVEMBER*Breeder: HiroiSex: 4 fem...,Koi,"['fall 2024', 'HIROI', 'koi', 'nisai', 'ochiba...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/00d04b3_HIROI_DOITSU_SANKE-SHOWA-TA...
3,564bc0a,8305083449513,IKARASHI MIX BOWL #1,600.0,False,https://gckoi.com/products/ikarashi-mix-bowl-1,*SHIP END OF NOVEMBER*Breeder: IkarashiSex: 5 ...,NaN,"['Beni Kikokuryu', 'fall 2024', 'ikarashi', 'k...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/564bc0a_IKARASHI_MIX_BOWL_#1.png
4,88ec869,7993174229161,ISA KOHAKU GUARANTEE FEMALE WITH CERTIFICATE 2...,2500.0,True,https://gckoi.com/products/isa-showa-grow-out-...,"Breeder: Isa with certificateSize: 21.5""Sex: f...",NaN,"['isa', 'kohaku', 'koi', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/88ec869_ISA_KOHAKU_GUARANTEE_FEMALE...
...,...,...,...,...,...,...,...,...,...,...,...
403,2fc5440,8479347048617,"TORAZO KOHAKU JUMBO TOSAI 9""-11"" WITH CERTIFIC...",400.0,False,https://gckoi.com/products/torazo-kohaku-jumbo...,Breeder: Torazo with certificate Sex: uncheckS...,NaN,"['kohaku', 'koi', 'torazo', 'tosai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/2fc5440_TORAZO_KOHAKU_JUMBO_TOSAI_9...
404,7b86441,8479347081385,"TORAZO KOHAKU JUMBO TOSAI 9""-11"" WITH CERTIFIC...",400.0,False,https://gckoi.com/products/torazo-kohaku-jumbo...,Breeder: Torazo with certificate Sex: uncheckS...,NaN,"['kohaku', 'koi', 'torazo', 'tosai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/7b86441_TORAZO_KOHAKU_JUMBO_TOSAI_9...
405,2e290d0,7864493572265,YAMAMATSU KOHAKU 17”-21” BOWL #1,450.0,False,https://gckoi.com/products/yamamatsu-sanke-17-...,"Breeder: Yamamatsu Size: 2sai 17”-21""Sex: uncheck",NaN,"['fall 2024', 'kohaku', 'koi', 'nisai', 'Yamam...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/2e290d0_YAMAMATSU_KOHAKU_17”-21”_BO...
406,cd8689c,8307867680937,YAMAMATSU KOHAKU 18”-19” BOWL #2,450.0,False,https://gckoi.com/products/yamamatsu-kohaku-18...,"Breeder: Yamamatsu Size: 2sai 18”-19""Sex: uncheck",NaN,"['fall 2024', 'kohaku', 'koi', 'nisai', 'Yamam...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/cd8689c_YAMAMATSU_KOHAKU_18”-19”_BO...


In [47]:
# check the full description for a few rows

for i in range(40):
  print(gckoi['description'][i])

Breeder: Genjiro Kohaku, Sanke and ShowaSize: 7”-9"Sex: uncheck
Breeder: MarukyuSize: 3sai 24”Sex: female
*SHIP END OF NOVEMBER*Breeder: HiroiSex: 4 females 3 malesSize: 2sai 14"-16"
*SHIP END OF NOVEMBER*Breeder: IkarashiSex: 5 Females 2 MalesSize: 16"-17"Age: 2sai
Breeder: Isa with certificateSize: 21.5"Sex: female
DEALER SELECT - SHIP FEBRUARY 2025Breeder: IzumiyaSize: 7”-8"Sex: uncheck
DEALER SELECT -  PHOTO AND VIDEO ARE SAMPLEBreeder: IzumiyaSize: 7”-8"Sex: uncheck
DEALER SELECT - SHIP FEBRUARY 2025Breeder: IzumiyaSize: 7”-8"Sex: uncheck
Breeder: IzumiyaSize: 7”-8"Sex: uncheck
Breeder: Kanno Metallic with some Doitsu GosankeSize: 5"-8"Sex: uncheck
*SHIP IN FEB 2025*SUPER CLEAN HIGH-QUALITY. DEALER SELECT FROM 35PCS IN THE VIDEOSize: 7”-9"  Sex: uncheck
DEALER SELECT - SHIP IN JANUARY 2025Breeder: Koshiji Tosai Mix: Doitsu Gosanke, Ginrin Showa, Kikusui, Yamato, Beni-Kikokuryu, Metalic Ochiba, Goshiki, Pearl Soragoi and a few KawarigoiSize: 1sai 7"-9"Sex: uncheck
Breeder: KOSHIJI 

- some are groups of fish, not just 1 fish. categorize by 1 fish or multiple fish by checking for '-' to indicate range in size
- Breeder is between "Breeder:" and "Sex" with no space before "Sex"
- Gender is after "Sex: ", but some are 'uncheck', so might be better to check for 'male' or 'female'
- some have "certificate", this could be a keyword to keep track of
- Size follows Size:

In [48]:
# prepare imports for using openai gpt models to help me separate the description into separate pieces of information

import openai
import os

In [49]:
# get my openai key loaded from os

OPENAI_API_KEY_LFZ = os.environ.get("OPENAI_API_KEY_LFZ")

# define the client

client = openai.OpenAI(api_key=OPENAI_API_KEY_LFZ)

In [50]:
# try using the model to organize the description

# set the system prompt

SYSTEM_PROMPT = {
  "role": "system",
  "content": """
You are an AI parser that extracts structured data from koi fish sale listings.

The user will provide a raw text description copied from an online koi fish selling website. Based on this description, extract the following fields and return them as a JSON object:

- "breeder": Name of the breeder (e.g., Dainichi, Sakai), or null if not mentioned.
- "length_inches": Length of the fish in inches (e.g., 7"-9"), or null if not mentioned.
- "gender": Gender of the fish, which must be either male or female. If not mentioned, or uncertain, return null.
- "certificate": Boolean. Indicate if a certificate is included. Null if unclear or not mentioned.
- "age": Age of the fish (e.g., "1 year", "Tosai", "2sai"), or null if not mentioned.
- "multiple_fish": Boolean. Set to true if the listing appears to describe multiple fish (e.g., mentions like "35pcs", "lot", "set of", or if the size is a range such as '7"-9"'), otherwise false.

**Important Instructions**:
- Always reply with a valid JSON object.
- Do not include any explanation or text outside the JSON.
- Use `null` for missing values.
- Be strict about format: no markdown, no backticks, just plain JSON.

Example input:
  "*SHIP IN FEB 2025*SUPER CLEAN HIGH-QUALITY. DEALER SELECT FROM 35PCS IN THE VIDEOSize: 7”-9\"  Sex: uncheck"

Expected output:
  {
    "breeder": null,
    "length_inches": "7-9",
    "gender": null,
    "certificate": null,
    "age": null,
    "multiple_fish": true
  }
"""}

conversation_history = [SYSTEM_PROMPT]

In [51]:
# start token_count

token_count = 0

In [52]:
# create a class in order to get structured response in the form of json

class ContentDict(BaseModel):
    breeder: str
    length_inches: str
    gender: str
    certificate: bool
    age: str
    multiple_fish: bool

In [53]:
# create a function to send the description to the LLM model to return dictionary of descriptors

import json

def send_description(description):
  # set token_count as global
  global token_count

  SYSTEM_PROMPT = {
  "role": "system",
  "content": """
  You are an AI parser that extracts structured data from koi fish sale listings.
  The user will provide a raw text description copied from an online koi fish selling website. Based on this description, extract the following fields and return them as a JSON object:

  - "breeder": Name of the breeder (e.g., Dainichi, Sakai), or null if not mentioned.
  - "length_inches": Length of the fish in inches (e.g., 7"-9"), or null if not mentioned.
  - "gender": Gender of the fish, which must be either male or female. If not mentioned, or uncertain, return null.
  - "certificate": Boolean. Indicate if a certificate is included. Null if unclear or not mentioned.
  - "age": Age of the fish (e.g., "1 year", "Tosai", "2sai"), or null if not mentioned.
  - "multiple_fish": Boolean. Set to true if the listing appears to describe multiple fish (e.g., mentions like "35pcs", "lot", "set of", or if the size is a range such as '7"-9"'), otherwise false.
  """}

  conversation_history = [SYSTEM_PROMPT]

  # add the description to the conversation history
  conversation_history.append({'role': 'user', 'content': description})

  # send the description to the model and get the response
  response = client.responses.parse(
    model='gpt-4.1-mini',
    input=conversation_history,
    temperature=0,
    text_format=ContentDict
  )

  # track the token usage
  token_count += response.usage.total_tokens

  # extract the content of the response (should be in JSON format)
  raw_content = response.output_text

  # check to see if the content is in JSON format, if so, parse it and return the dictionary
  try:
      content_dict = json.loads(raw_content)
  # if there is an error, then print the error
  except json.JSONDecodeError:
      print("Model did not return valid JSON.")
      print("Raw response:", raw_content)
      content_dict = {}

  return content_dict

In [54]:
# test the function #1

test_description = gckoi['description'][100]
test_description

'Breeder: Momotaro\xa0Size: tosai 7"-9"Sex: unknown\xa0These are "new type" with Karashi bloodline. Some 2sai can reach 26"-30"'

In [55]:
test_result = send_description(test_description)
test_result

{'breeder': 'Momotaro',
 'length_inches': '7"-9"',
 'gender': '',
 'certificate': False,
 'age': 'tosai',
 'multiple_fish': True}

In [56]:
token_count

431

In [57]:
# test the function #2

test_description = gckoi['description'][85]
test_description

'Breeder: MARUSEISize: 7"-9"Sex: uncheck*WILL BE RANDOMLY SELECTED. LIMITED ONLY 40PCS'

In [58]:
test_result = send_description(test_description)
test_result

{'breeder': 'MARUSEI',
 'length_inches': '7"-9"',
 'gender': '',
 'certificate': False,
 'age': '',
 'multiple_fish': True}

In [59]:
# test the function #3

test_description = gckoi['description'][200]
test_description

'Breeder:\xa0KansukeSex: 80% female\xa0Size: 2sai\xa017”-19”'

In [60]:
test_result = send_description(test_description)
test_result

{'breeder': 'Kansuke',
 'length_inches': '17”-19”',
 'gender': '',
 'certificate': False,
 'age': '2sai',
 'multiple_fish': True}

In [61]:
token_count

1260

In [62]:
# test the function on a few rows first

test_gckoi = gckoi.head(5).copy()
test_gckoi

,uuid,sku,title,price,in_stock,link,description,category,tags,image_url,image_path
0,9718985,8428729860265,"GENJIRO GOSANKE 7""-9""",95.0,True,https://gckoi.com/products/genjiro-gosanke-7-9,"Breeder: Genjiro Kohaku, Sanke and ShowaSize: ...",NaN,"['genjiro', 'kohaku', 'koi', 'sanke', 'showa',...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/9718985_GENJIRO_GOSANKE_7-9.jpg
1,d9a7dbc,7856409346217,HIGH QUALITY SAKAI BLOODLINE TANCHO KOHAKU 24”...,3500.0,True,https://gckoi.com/products/high-quality-sakai-...,Breeder: MarukyuSize: 3sai 24”Sex: female,NaN,"['kohaku', 'koi', 'Marukyu', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/d9a7dbc_HIGH_QUALITY_SAKAI_BLOODLIN...
2,00d04b3,8291399860393,"HIROI DOITSU SANKE-SHOWA-TANCHO OCHIBA 14""-16""",600.0,False,https://gckoi.com/products/hiroi-doitsu-sanke-...,*SHIP END OF NOVEMBER*Breeder: HiroiSex: 4 fem...,Koi,"['fall 2024', 'HIROI', 'koi', 'nisai', 'ochiba...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/00d04b3_HIROI_DOITSU_SANKE-SHOWA-TA...
3,564bc0a,8305083449513,IKARASHI MIX BOWL #1,600.0,False,https://gckoi.com/products/ikarashi-mix-bowl-1,*SHIP END OF NOVEMBER*Breeder: IkarashiSex: 5 ...,NaN,"['Beni Kikokuryu', 'fall 2024', 'ikarashi', 'k...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/564bc0a_IKARASHI_MIX_BOWL_#1.png
4,88ec869,7993174229161,ISA KOHAKU GUARANTEE FEMALE WITH CERTIFICATE 2...,2500.0,True,https://gckoi.com/products/isa-showa-grow-out-...,"Breeder: Isa with certificateSize: 21.5""Sex: f...",NaN,"['isa', 'kohaku', 'koi', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/88ec869_ISA_KOHAKU_GUARANTEE_FEMALE...


In [63]:
# apply the function on the test df

# track the progress

from tqdm import tqdm

tqdm.pandas()

test_gckoi['gpt_dict'] = test_gckoi['description'].progress_apply(send_description)
test_gckoi['gpt_dict']

100%|██████████| 5/5 [00:06<00:00,  1.25s/it]


0    {'breeder': 'Genjiro', 'length_inches': '7”-9"...
1    {'breeder': 'Marukyu', 'length_inches': '24"',...
2    {'breeder': 'Hiroi', 'length_inches': '14"-16"...
3    {'breeder': 'Ikarashi', 'length_inches': '16"-...
4    {'breeder': 'Isa', 'length_inches': '21.5"', '...
Name: gpt_dict, dtype: object

In [64]:
# check the full dictionary

for item in test_gckoi['gpt_dict']:
  print(item)

{'breeder': 'Genjiro', 'length_inches': '7”-9"', 'gender': '', 'certificate': False, 'age': '', 'multiple_fish': True}
{'breeder': 'Marukyu', 'length_inches': '24"', 'gender': 'female', 'certificate': False, 'age': '3sai', 'multiple_fish': False}
{'breeder': 'Hiroi', 'length_inches': '14"-16"', 'gender': '', 'certificate': False, 'age': '2sai', 'multiple_fish': True}
{'breeder': 'Ikarashi', 'length_inches': '16"-17"', 'gender': '', 'certificate': False, 'age': '2sai', 'multiple_fish': True}
{'breeder': 'Isa', 'length_inches': '21.5"', 'gender': 'female', 'certificate': True, 'age': '', 'multiple_fish': False}


In [65]:
# make sure the results can be turned into dataframe without any issue

test_df_1 = pd.json_normalize(test_gckoi['gpt_dict'])
test_df_1.index = test_gckoi['uuid']
test_df_1

,breeder,length_inches,gender,certificate,age,multiple_fish
uuid,,,,,,
9718985,Genjiro,"7”-9""",,False,,True
d9a7dbc,Marukyu,"24""",female,False,3sai,False
00d04b3,Hiroi,"14""-16""",,False,2sai,True
564bc0a,Ikarashi,"16""-17""",,False,2sai,True
88ec869,Isa,"21.5""",female,True,,False


In [66]:
# check the token count

token_count

3316

In [67]:
# calculate the number of tokens used for each row

before_tokens = 1260
after_tokens = 3316
tokens_used_for_5_rows = after_tokens - before_tokens
token_per_row = tokens_used_for_5_rows / 5
token_per_row

411.2

In [68]:
# check the token count before applying the function to the entire dataframe

token_count

3316

In [69]:
# apply the function on the entire df
# store result row by row instead of as 1 big operation, so the result can be saved even if something interrupts the api call

# track the progress
from tqdm import tqdm
import time

# create the column if it does not exist
if 'gpt_dict' not in gckoi.columns:
    gckoi['gpt_dict'] = None

for i, row in tqdm(gckoi.iterrows(), total=len(gckoi)):
  # skip the row if already done, so that it can be resumable
  if pd.isnull(row.get('gpt_dict')):
    try:
      result = send_description(row['description'])
    except Exception as e:
      print(f"Row {i} failed: {e}")
      result = {}
    gckoi.at[i, 'gpt_dict'] = result

  # save partial progress
  if i % 10 == 0:
    gckoi.to_parquet('gckoi_partial.parquet')
    time.sleep(1)

 53%|█████▎    | 215/408 [06:18<04:22,  1.36s/it]

Row 215 failed: Out of range float values are not JSON compliant


 70%|███████   | 286/408 [08:52<05:21,  2.64s/it]

Row 286 failed: Out of range float values are not JSON compliant
Row 287 failed: Out of range float values are not JSON compliant
Row 288 failed: Out of range float values are not JSON compliant
Row 289 failed: Out of range float values are not JSON compliant


 94%|█████████▎| 382/408 [11:18<00:38,  1.49s/it]

Row 382 failed: Out of range float values are not JSON compliant


100%|██████████| 408/408 [11:54<00:00,  1.75s/it]


In [70]:
gckoi['gpt_dict']

0      {'breeder': 'Genjiro', 'length_inches': '7”-9"...
1      {'breeder': 'Marukyu', 'length_inches': '24"',...
2      {'breeder': 'Hiroi', 'length_inches': '14"-16"...
3      {'breeder': 'Ikarashi', 'length_inches': '16"-...
4      {'breeder': 'Isa', 'length_inches': '21.5"', '...
                             ...                        
403    {'breeder': 'Torazo', 'length_inches': '10"-12...
404    {'breeder': 'Torazo', 'length_inches': '10"-12...
405    {'breeder': 'Yamamatsu', 'length_inches': '17”...
406    {'breeder': 'Yamamatsu', 'length_inches': '18”...
407    {'breeder': 'Yamamatsu', 'length_inches': '17"...
Name: gpt_dict, Length: 408, dtype: object

In [71]:
# save the dataframe to parquet

gckoi.to_parquet('gckoi_gpt_dict_done.parquet')

In [72]:
# turn the dictionary into a dataframe of its own, add the uuid as the index, to later merge on

gckoi_gpt_dict = pd.json_normalize(gckoi['gpt_dict'])
gckoi_gpt_dict.index = gckoi['uuid']
gckoi_gpt_dict

,breeder,length_inches,gender,certificate,age,multiple_fish
uuid,,,,,,
9718985,Genjiro,"7”-9""",,False,,True
d9a7dbc,Marukyu,"24""",female,False,3sai,False
00d04b3,Hiroi,"14""-16""",,False,2sai,True
564bc0a,Ikarashi,"16""-17""",,False,2sai,True
88ec869,Isa,"21.5""",female,True,,False
...,...,...,...,...,...,...
2fc5440,Torazo,"10""-12""",,True,,True
7b86441,Torazo,"10""-12""",,True,,True
2e290d0,Yamamatsu,"17”-21""",,False,2sai,True


In [73]:
# merge the gpt_dict column onto the main gckoi dataframe, on uuid

gckoi_merged = pd.merge(gckoi, gckoi_gpt_dict, how='left', on='uuid')
gckoi_merged

,uuid,sku,title,price,in_stock,link,description,category,tags,image_url,image_path,gpt_dict,breeder,length_inches,gender,certificate,age,multiple_fish
0,9718985,8428729860265,"GENJIRO GOSANKE 7""-9""",95.0,True,https://gckoi.com/products/genjiro-gosanke-7-9,"Breeder: Genjiro Kohaku, Sanke and ShowaSize: ...",NaN,"['genjiro', 'kohaku', 'koi', 'sanke', 'showa',...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/9718985_GENJIRO_GOSANKE_7-9.jpg,"{'breeder': 'Genjiro', 'length_inches': '7”-9""...",Genjiro,"7”-9""",,False,,True
1,d9a7dbc,7856409346217,HIGH QUALITY SAKAI BLOODLINE TANCHO KOHAKU 24”...,3500.0,True,https://gckoi.com/products/high-quality-sakai-...,Breeder: MarukyuSize: 3sai 24”Sex: female,NaN,"['kohaku', 'koi', 'Marukyu', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/d9a7dbc_HIGH_QUALITY_SAKAI_BLOODLIN...,"{'breeder': 'Marukyu', 'length_inches': '24""',...",Marukyu,"24""",female,False,3sai,False
2,00d04b3,8291399860393,"HIROI DOITSU SANKE-SHOWA-TANCHO OCHIBA 14""-16""",600.0,False,https://gckoi.com/products/hiroi-doitsu-sanke-...,*SHIP END OF NOVEMBER*Breeder: HiroiSex: 4 fem...,Koi,"['fall 2024', 'HIROI', 'koi', 'nisai', 'ochiba...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/00d04b3_HIROI_DOITSU_SANKE-SHOWA-TA...,"{'breeder': 'Hiroi', 'length_inches': '14""-16""...",Hiroi,"14""-16""",,False,2sai,True
3,564bc0a,8305083449513,IKARASHI MIX BOWL #1,600.0,False,https://gckoi.com/products/ikarashi-mix-bowl-1,*SHIP END OF NOVEMBER*Breeder: IkarashiSex: 5 ...,NaN,"['Beni Kikokuryu', 'fall 2024', 'ikarashi', 'k...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/564bc0a_IKARASHI_MIX_BOWL_#1.png,"{'breeder': 'Ikarashi', 'length_inches': '16""-...",Ikarashi,"16""-17""",,False,2sai,True
4,88ec869,7993174229161,ISA KOHAKU GUARANTEE FEMALE WITH CERTIFICATE 2...,2500.0,True,https://gckoi.com/products/isa-showa-grow-out-...,"Breeder: Isa with certificateSize: 21.5""Sex: f...",NaN,"['isa', 'kohaku', 'koi', 'nisai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/88ec869_ISA_KOHAKU_GUARANTEE_FEMALE...,"{'breeder': 'Isa', 'length_inches': '21.5""', '...",Isa,"21.5""",female,True,,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403,2fc5440,8479347048617,"TORAZO KOHAKU JUMBO TOSAI 9""-11"" WITH CERTIFIC...",400.0,False,https://gckoi.com/products/torazo-kohaku-jumbo...,Breeder: Torazo with certificate Sex: uncheckS...,NaN,"['kohaku', 'koi', 'torazo', 'tosai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/2fc5440_TORAZO_KOHAKU_JUMBO_TOSAI_9...,"{'breeder': 'Torazo', 'length_inches': '10""-12...",Torazo,"10""-12""",,True,,True
404,7b86441,8479347081385,"TORAZO KOHAKU JUMBO TOSAI 9""-11"" WITH CERTIFIC...",400.0,False,https://gckoi.com/products/torazo-kohaku-jumbo...,Breeder: Torazo with certificate Sex: uncheckS...,NaN,"['kohaku', 'koi', 'torazo', 'tosai']",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/7b86441_TORAZO_KOHAKU_JUMBO_TOSAI_9...,"{'breeder': 'Torazo', 'length_inches': '10""-12...",Torazo,"10""-12""",,True,,True
405,2e290d0,7864493572265,YAMAMATSU KOHAKU 17”-21” BOWL #1,450.0,False,https://gckoi.com/products/yamamatsu-sanke-17-...,"Breeder: Yamamatsu Size: 2sai 17”-21""Sex: uncheck",NaN,"['fall 2024', 'kohaku', 'koi', 'nisai', 'Yamam...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/2e290d0_YAMAMATSU_KOHAKU_17”-21”_BO...,"{'breeder': 'Yamamatsu', 'length_inches': '17”...",Yamamatsu,"17”-21""",,False,2sai,True
406,cd8689c,8307867680937,YAMAMATSU KOHAKU 18”-19” BOWL #2,450.0,False,https://gckoi.com/products/yamamatsu-kohaku-18...,"Breeder: Yamamatsu Size: 2sai 18”-19""Sex: uncheck",NaN,"['fall 2024', 'kohaku', 'koi', 'nisai', 'Yamam...",https://cdn.shopify.com/s/files/1/0565/8830/45...,gckoi_data/cd8689c_YAMAMATSU_KOHAKU_18”-19”_BO...,"{'breeder': 'Yamamatsu', 'length_inches': '18”...",Yamamatsu,"18”-19""",,False,2sai,True


In [74]:
# export merged df to csv

gckoi_merged.to_csv('gckoi_df.csv', index=False)

In [75]:
# export the dataframe to json to maintain list formats

with open('gckoi.json', 'w') as f:
  json.dump(gckoi_merged.to_dict(orient='records'), f)